In [2]:
import numpy as np
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import datetime
import scipy
import sklearn

In [3]:
print(tf.config.list_physical_devices('GPU'))

[]


In [4]:
IMPORT_COUNT = 5000000
TEST_COUNT = 10000

In [5]:
# Generate random seed
#myrand=np.random.randint(1, 99999 + 1)
myrand=71926
np.random.seed(myrand)
tf.random.set_seed(myrand)
print("Random seed is:",myrand)

Random seed is: 71926


In [6]:
RNG_OUTPUT_FILENAME="mersenne_twist_xorshifter.txt"
df = np.genfromtxt(RNG_OUTPUT_FILENAME,delimiter=', ',dtype='uint64', max_rows=IMPORT_COUNT)

In [7]:
#calculates how many bits are in the output.
BIT_WIDTH=np.ceil(np.log2(np.amax(df))).astype(int)
# convert the generated numbers to binary sequences
df_as_bits = np.concatenate(((df[:, 0][:,None] & (1 << np.arange(BIT_WIDTH,dtype='uint64')) > 0).astype(int), (df[:, 1][:,None] & (1 << np.arange(BIT_WIDTH,dtype='uint64')) > 0).astype(int)), axis=1)

In [8]:
# shuffle the data frames
indicies = np.arange(df_as_bits.shape[0],dtype='uint64')
np.random.shuffle(indicies)
df_as_bits=df_as_bits[indicies]

In [9]:
# convert the data into inputs and outputs
y = df_as_bits[:,32:]
X = df_as_bits[:,:32]

In [10]:
# Split the data into train and test data
X_train = X[TEST_COUNT:]
X_test = X[:TEST_COUNT]
y_train = y[TEST_COUNT:]
y_test = y[:TEST_COUNT]

In [11]:
def build_model(hp):
    LOSS="binary_crossentropy"
    model = Sequential()
    model.add(Dense(640, activation='relu',input_shape=[X.shape[1]] ))
    model.add(Dense(y.shape[1], activation='sigmoid'))
    
    opt = keras.optimizers.Nadam(
        learning_rate=hp.Float("learning_rate", 10**(-5), 10**(-2),sampling="log"),
        epsilon=hp.Float("epsilon",1e-7,1e-4,sampling="log"),
        beta_1=hp.Float("beta_1",.8,.99999,sampling="reverse_log"),
        beta_2=hp.Float("beta_2",.8,.99999,sampling="reverse_log"),
        )
    model.compile(optimizer=opt, loss=LOSS,metrics=['binary_accuracy'])
    model.summary()
    return model

In [12]:
#define callback functions
stopEarly = tf.keras.callbacks.EarlyStopping(
    monitor='val_binary_accuracy', min_delta=.001, patience=10, verbose=0, mode='auto', restore_best_weights=False
)

log_dir = "tmp/hyperparameters_tempering/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,profile_batch=0)

In [13]:
# extract a short set from the training for hyper parameter tuning
X_train_short= X_train[:1000000]
y_train_short= y_train[:1000000]

In [14]:
%%time
# Tuning the hyper parameters
tuner = kt.tuners.bayesian.BayesianOptimization(build_model,'val_binary_accuracy',25,project_name="tmp/tempering", seed=myrand)

while tuner.remaining_trials>0:
    try:
        tuner.search(X_train_short, y_train_short,batch_size=2048, epochs=100, validation_data=(X_test,y_test),callbacks=[stopEarly,tensorboard_callback])
    except:
        pass

tuner.results_summary()

Trial 25 Complete [00h 07m 46s]
val_binary_accuracy: 0.9846875071525574

Best val_binary_accuracy So Far: 1.0
Total elapsed time: 02h 32m 18s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in .\tmp/tempering
Showing 10 best trials
Trial summary
Hyperparameters:
learning_rate: 0.008718330030541725
epsilon: 5.751763055109973e-06
beta_1: 0.8989345278917303
beta_2: 0.9074604423190239
Score: 1.0
Trial summary
Hyperparameters:
learning_rate: 0.0038593516478438684
epsilon: 7.563276785784439e-07
beta_1: 0.8822897791929948
beta_2: 0.8845332809911142
Score: 0.9999468922615051
Trial summary
Hyperparameters:
learning_rate: 0.01
epsilon: 7.632069047112443e-06
beta_1: 0.9099285589191526
beta_2: 0.8000000000000002
Score: 0.9998999834060669
Trial summary
Hyperparameters:
learning_rate: 0.01
epsilon: 4.38555603367085e-06
beta_1: 0.8000000000000002
beta_2: 0.8847060147913317
Score: 0.9881437420845032
Trial summary
Hyperparameters:
learning_rate: 0.001950816046384373
epsilon: 2.37678583129

In [15]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
# use the best model for training
model = tuner.hypermodel.build(best_hps)

best_hps.values

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 640)               21120     
                                                                 
 dense_3 (Dense)             (None, 32)                20512     
                                                                 
Total params: 41,632
Trainable params: 41,632
Non-trainable params: 0
_________________________________________________________________


{'learning_rate': 0.008718330030541725,
 'epsilon': 5.751763055109973e-06,
 'beta_1': 0.8989345278917303,
 'beta_2': 0.9074604423190239}

In [17]:
def train_model(model, X = X_train, Y=y_train, epochs=100, batch_size=2048,verbose=1, log_dir = "tmp/dense_model/"):
    log_dir += datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,profile_batch=0)
    model.fit(X, Y, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size,callbacks=[tensorboard_callback],verbose=verbose)
    return model

In [ ]:
%%time
my_model_trained = train_model(model, epochs=30,log_dir = "tmp/mt_tempering/")

Epoch 1/30
2437/2437 [==============================] - 7s 3ms/step - loss: 0.4978 - binary_accuracy: 0.6624 - val_loss: 0.3570 - val_binary_accuracy: 0.7718
Epoch 2/30
2437/2437 [==============================] - 7s 3ms/step - loss: 0.2782 - binary_accuracy: 0.8192 - val_loss: 0.2282 - val_binary_accuracy: 0.8524
Epoch 3/30
2437/2437 [==============================] - 7s 3ms/step - loss: 0.1687 - binary_accuracy: 0.9019 - val_loss: 0.1136 - val_binary_accuracy: 0.9372
Epoch 4/30
2437/2437 [==============================] - 7s 3ms/step - loss: 0.0890 - binary_accuracy: 0.9470 - val_loss: 0.0756 - val_binary_accuracy: 0.9517
Epoch 5/30
2437/2437 [==============================] - 7s 3ms/step - loss: 0.0673 - binary_accuracy: 0.9567 - val_loss: 0.0559 - val_binary_accuracy: 0.9656
Epoch 6/30
2437/2437 [==============================] - 7s 3ms/step - loss: 0.0514 - binary_accuracy: 0.9670 - val_loss: 0.0483 - val_binary_accuracy: 0.9677
Epoch 7/30
2437/2437 [==============================

In [ ]:
results = model.evaluate(X_test, y_test, batch_size=128)
print("test loss: %f, test acc: %s" % tuple(results))

79/79 [==============================] - 0s 1ms/step - loss: 6.3016e-05 - binary_accuracy: 1.0000
test loss: 0.000063, test acc: 1.0


In [18]:
model.save("mt_tempering_model.h5")